In [1]:
# rag_ollama_chroma.py
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import os
from pathlib import Path

In [2]:
root_path = Path(os.getcwd()).parent.parent
ollama_model = root_path / "ollama-service" / "models" / "ko_llama" / "llama-3-Korean-Bllossom-8B.Q4_K_M.gguf"
embeddings_path = root_path / "ollama-service" / "models" / "BGE-m3-ko"
print(ollama_model)
print(embeddings_path)

f:\project_test\solutions\chat\test1_1\ollama-service\models\ko_llama\llama-3-Korean-Bllossom-8B.Q4_K_M.gguf
f:\project_test\solutions\chat\test1_1\ollama-service\models\BGE-m3-ko


In [3]:
# HuggingFace 임베딩 모델 로드 (경로를 문자열로 변환하여 오류 방지)
embedding_model = HuggingFaceEmbeddings(
    model_name=str(embeddings_path)
)
# llm = Ollama(model=str(ollama_model))

f:\project_test\solutions\chat\.venv_chat\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
vectorstore = FAISS.load_local("vector_db/pcn_web", embedding_model, allow_dangerous_deserialization=True)

In [5]:
query_embedding = embedding_model.embed_query("피씨엔 소개")

# 벡터 유사도 검색 (상위 3개 결과 반환)
results = vectorstore.similarity_search_by_vector(query_embedding, k=20)

f:\project_test\solutions\chat\.venv_chat\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [10]:
# 검색기(Retriever) 설정 (개선)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})  
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = ChatOllama(
    model=str("ko-llama-8B"),
    temperature=0.1,  # 약간의 다양성 부여
    base_url="http://localhost:11434",
    callback_manager=callback_manager,
    retriever=retriever  # retriever를 llm에 직접 설정
)

# 프롬프트를 더 명확하고 자연스럽게 개선
system_template = (
    '''
    당신은 친절하고 유능한 AI 어시스턴트입니다.
    사용자의 질문에 대해 신뢰할 수 있는 정보를 바탕으로 정확하고 간결하게 답변하세요.
    주어진 정보를 바탕으로 답변하세요.
    **피씨엔**, 또는 **PCN**이라는 정보를 입력받으면 피씨엔(기술), PCN 회사 소개를 답변하세요.
    피씨엔, PCN 회사 소개는 다음과 같습니다.
    피씨엔(PCN)은 기술 회사로 빅데이터, XR, AI, SI 등 다양한 서비스를 제공합니다.
    회사개요, 프로젝트(Project), 핵심 역량 및 사업 분야, 미션, 비전, 가치, 주요실적 및 프로젝트, 조직 규모 및 연혁 등 회사 정보를 답변하세요.
    답변에서 회사명은 PCN으로 표기하세요.
    '''
)
human_template = (
    "아래의 질문에 대해 단계별로 논리적으로 생각하여 답변해 주세요.\n"
    "질문: {question}"
)

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = prompt | llm

In [ ]:
chain.invoke({"question": "피씨엔 회사 소개"})